## Check List 7.1. Demo - White board sweeping
* In this demo file, we will use two robots (Indy7, Panda) to sweep a white board, removing obstacles

* You need trained model to use ReachChecker
  - model/reach_svm/indy7.json
  - model/reach_svm/panda.json
  - 백업: 개인 이동식 하드디스크, 강준수

### Prepare the task scene
* Prepare Indy7 and panda, and install sweeping tool to indy. (Check release/Figs/7.1.WhiteBoardSweeping.jpg)
* Prepare "floor", "track", "box1", "box2" objects as defined in pkg.detector.aruco.marker_config.py
* The robots and "track" are installed on "floor".
* "box1" and "box2" are on "track"
<img src="../Figs/7.1.WhiteBoardSweeping.jpg" width="80%">

## set running directory to project source

In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))
from pkg.utils.code_scraps import add_indy_sweep_tool, \
        use_current_place_point_only, use_current_sub_binders_only, \
        finish_L_shape, set_l_shape_object, ModeSwitcher, double_sweep_motions

## init combined robot config

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, ((0.3,-0.4,0), (0,0,np.pi/2)),
                INDY_IP),
    RobotConfig(1, RobotType.panda, ((-0.3,-0.4,0), (0,0,np.pi/2)),
                "{}/{}".format(PANDA_REPEATER_IP, PANDA_ROBOT_IP))]
              , connection_list=[False, False])

connection command:
indy0: False
panda1: False


In [3]:
from pkg.detector.aruco.marker_config import *
from pkg.detector.aruco.stereo import ArucoStereo
from pkg.detector.camera.realsense import RealSense
from pkg.detector.camera.kinect import Kinect
aruco_map = get_aruco_map()
stereo = ArucoStereo(aruco_map, [Kinect(), RealSense()])
stereo.initialize()
stereo.calibrate()

Device configuration: 
	color_format: 3 
	(0:JPG, 1:NV12, 2:YUY2, 3:BGRA32)

	color_resolution: 5 
	(0:OFF, 1:720p, 2:1080p, 3:1440p, 4:1536p, 5:2160p, 6:3072p)

	depth_mode: 3 
	(0:OFF, 1:NFOV_2X2BINNED, 2:NFOV_UNBINNED,3:WFOV_2X2BINNED, 4:WFOV_UNBINNED, 5:Passive IR)

	camera_fps: 2 
	(0:5 FPS, 1:15 FPS, 2:30 FPS)

	synchronized_images_only: False 
	(True of False). Drop images if the color and depth are not synchronized

	depth_delay_off_color_usec: 0 ms. 
	Delay between the color image and the depth image

	wired_sync_mode: 0
	(0:Standalone mode, 1:Master mode, 2:Subordinate mode)

	subordinate_delay_off_master_usec: 0 ms.
	The external synchronization timing.

	disable_streaming_indicator: False 
	(True or False). Streaming indicator automatically turns on when the color or depth camera's are in use.


Start streaming


[(array([[1.82983423e+03, 0.00000000e+00, 1.91572046e+03],
         [0.00000000e+00, 1.82983423e+03, 1.09876086e+03],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  array([ 7.09966481e-01, -2.73409390e+00,  1.45804870e-03, -3.24774766e-04,
          1.44911301e+00,  5.84310412e-01, -2.56374550e+00,  1.38472950e+00])),
 (array([[1.39560388e+03, 0.00000000e+00, 9.62751587e+02],
         [0.00000000e+00, 1.39531934e+03, 5.47687012e+02],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  array([0., 0., 0., 0., 0.])),
 array([[ 0.8257667 , -0.02568319, -0.5634268 ,  0.6506724 ],
        [ 0.17509305,  0.96127963,  0.21280013, -0.16074805],
        [ 0.53614527, -0.27437538,  0.79828966,  0.09146631],
        [ 0.        ,  0.        ,  0.        ,  1.        ]],
       dtype=float32)]

## create scene builder

In [4]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(stereo)
s_builder.reset_reference_coord(ref_name="floor")

## detect robot and make geometry scene

In [5]:
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
# save_pickle("data/xyz_rpy_robots.pkl", xyz_rpy_robots)
xyz_rpy_robots = load_pickle("data/xyz_rpy_robots.pkl")
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)

Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## init planning pipeline

In [6]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)


```
open web ui on <your ip>:8050
click geometry items / Handles / Binders to highlight geometry on RVIZ
other functions may be buggy.. please report
```

## add environment

In [7]:
gtem_dict = load_pickle("data/ENVIRONMENT.pkl")
gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.ENVIRONMENT], xyz_rpy_dict=gtem_dict)
# save_pickle("data/ENVIRONMENT.pkl", gtem_dict)


# add cam poles
ptems = s_builder.add_poles({"cam0": s_builder.ref_coord_inv[:3,3], "cam1":np.matmul(s_builder.ref_coord_inv, stereo.T_c12)[:3,3]},
                            color=(0.6,0.6,0.6,0.0))
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
gscene.NAME_DICT["floor"].dims = (2,1,0.01)
gscene.set_workspace_boundary( -1, 1, -0.5, 0.5, -0.1, 1.05)

 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
Please create a subscriber to the marker


## add components

In [8]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepFramer, FixtureSlot
from pkg.planning.constraint.constraint_subject import \
        CustomObject, Grasp2Point, PlacePoint, SweepFrame, SweepTask, BoxObject, FixturePoint, AbstractObject

from pkg.planning.constraint.constraint_common import MotionConstraint
from pkg.planning.constraint.constraint_subject import AbstractTask
from pkg.planning.constraint.constraint_subject import SweepLineTask

add_indy_sweep_tool(gscene, "indy0", face_name="brush_face", tool_offset=0.115)

gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip1", link_name="panda1_hand", 
                 dims=(0.01,)*3, center=(0,0,0.112), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

pscene.create_binder(bname="grip1", gname="grip1", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="brush_face", gname="brush_face", _type=SweepFramer, point=(0,0,gscene.NAME_DICT['brush_face'].dims[2]/2), 
                     rpy=(0,np.pi,0))

track = s_builder.detect_and_register(item_names=["track"], level_mask=[DetectionLevel.ENVIRONMENT])["track"]
track_face = gscene.copy_from(track, new_name="track_face", collision=False)
track_face.dims = (track.dims[0]-0.1, track.dims[1]-0.15, track.dims[2])
pscene.create_binder(bname="track_face", gname="track_face", _type=PlacePlane, point=None)
# track.dims = (track.dims[0]-0.32, track.dims[1]-0.12, track.dims[2])

gscene.update_markers_all()

TOOL_DIM = [0.06, 0.10]
TACK_WIDTH = 0.15
TRACK_DIM = track_face.dims
TRACK_NUM = np.ceil(np.divide(TRACK_DIM[1]-TOOL_DIM[1], TOOL_DIM[1])).astype(np.int)+1
TRACK_STEP = (TRACK_DIM[1]-TOOL_DIM[1])/(TRACK_NUM-1)
WP_REF_A = -np.subtract(TRACK_DIM[:2], TOOL_DIM[:2])/2
WP_REF_B = np.array([-WP_REF_A[0], WP_REF_A[1]])
WP_REF_M = np.array([0, WP_REF_A[1]])
TRC_THIC = TRACK_DIM[2]
track_list = []
for i_trc in range(TRACK_NUM):
    wp1 = gscene.create_safe(GEOTYPE.BOX, "wp{}a".format(i_trc+1), "base_link", (TOOL_DIM[0]/2,TOOL_DIM[1]/2,TRC_THIC), 
                             tuple(WP_REF_A+[0,TRACK_STEP*i_trc])+(0,),rpy=(0,0,0), 
                             color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="track_face")
    wp2 = gscene.create_safe(GEOTYPE.BOX, "wp{}b".format(i_trc+1), "base_link", (TOOL_DIM[0]/2,TOOL_DIM[1]/2,TRC_THIC), 
                             tuple(WP_REF_B+[0,TRACK_STEP*i_trc])+(0,),rpy=(0,0,0), 
                             color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="track_face")
    face = gscene.create_safe(GEOTYPE.BOX, "face{}".format(i_trc+1), "base_link", 
                              (TRACK_DIM[0] - TOOL_DIM[0]/2,TACK_WIDTH,TRC_THIC), 
                             center=tuple(WP_REF_M+[0,TRACK_STEP*i_trc])+(1e-2,),rpy=(0,0,0), 
                             color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="track_face")
    track_list.append((wp1, wp2, face))
    
    gscene.update_markers_all()
    
for gname in sorted(gtem_dict.keys()):
    pscene.create_subject(oname=gname, gname=gname, _type=BoxObject, hexahedral=True)
# box3= pscene.create_subject(oname="box3", gname="box3", _type=BoxObject, hexahedral=True)

sweep_list = []
for i_t, track_tem in enumerate(track_list):
    wp1, wp2, face = track_tem
    sweep_ = pscene.create_subject(oname="sweep{}".format(i_t+1), gname="track_face", _type=SweepLineTask, 
                                   action_points_dict = {wp1.name: SweepFrame(wp1.name, wp1, [0,0,0.005], [0,0,0]),
                                                       wp2.name: SweepFrame(wp2.name, wp2, [0,0,0.005], [0,0,0])}, 
                                   clearance=[face])
    sweep_list.append(sweep_)
    
    gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])
gscene.update_markers_all()

# ## add blocking structure on table
# gscene.create_safe(gtype=GEOTYPE.BOX, name="blocker", link_name="base_link",
#                             center=(-0.27,-0.2,0.15), dims=(0.1,0.3, 0.4), rpy=(0,0,0), color=(0.0,0.8,0.0,0.5),
#                             collision=True, fixed=True)

### planners

In [9]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
mplan.update_gscene()
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker
from pkg.planning.filtering.task_clearance_filter import TaskClearanceChecker

gcheck = GraspChecker(pscene)
rcheck = ReachChecker(pscene)
tcheck = TaskClearanceChecker(pscene, gcheck)
checkers_all = [tcheck, rcheck, gcheck]
# lcheck = LatticedChecker(pscene, gcheck)
# checkers_all.append(lcheck)
mplan.motion_filters = checkers_all
gscene.show_pose(crob.home_pose)

from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy
gtimer = GlobalTimer.instance()
initial_state = pscene.initialize_state(crob.home_pose)
print(initial_state.node)

# remove place points except for the current one
use_current_place_point_only(pscene, initial_state)

pscene.subject_name_list

('track_face', 0, 0, 0, 'track_face', 'track_face')


['floor', 'sweep1', 'sweep2', 'sweep3', 'track', 'wall']

# CustomRule

In [10]:
from pkg.planning.task.custom_rules.sweep_entrance_control import SweepEntranceControlRule
from pkg.planning.constraint.constraint_common import get_binding_margins, fit_binding
from pkg.utils.traj_utils import simplify_schedule, mix_schedule
tplan.custom_rule = SweepEntranceControlRule(pscene)


# ## sampler
# from pkg.planning.sampling.node_sampling import make_state_param_hashable, UniformNodeSampler, PenaltyNodeSampler, GrowingSampler

# tplan.new_node_sampler = PenaltyNodeSampler(3, 1)
# tplan.parent_node_sampler = UniformNodeSampler(3)
# # tplan.parent_snode_sampler = GrowingSampler(3)

## Detect once

In [11]:
mplan.reset_log(False)
gtimer.reset()

with gtimer.block("detect"):
    for sname in pscene.subject_name_list:
        if isinstance(pscene.subject_dict[sname], AbstractObject):
            pscene.remove_subject(sname)
            
    gscene.clear_non_fixed()
    
    ## detect again
    gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])

    ## create boxes
    box_list =  [] 
    for gname in sorted(gtem_dict.keys()):
        box_new = pscene.create_subject(oname=gname, gname=gname, _type=BoxObject, hexahedral=True)
        box_new.register_binders(pscene, PlacePlane)
        box_list.append(box_new)
    
    initial_state = pscene.initialize_state(crob.home_pose, force_fit_binding=True)
    gscene.update_markers_all()

    # remove place points and sub-binders except for the current ones
    use_current_place_point_only(pscene, initial_state)
    use_current_sub_binders_only(pscene, initial_state)
    tplan.prepare()
    mplan.update_gscene()

## Plan & execute

In [12]:
from pkg.planning.constraint.constraint_common import get_binding_margins, fit_binding
crob.reset_connection(True,True)

connection command:
indy0: True
panda1: True
Connect: Server IP (192.168.0.63)


### Test gripper

In [14]:
crob.grasp(True, True)
time.sleep(0.5)
crob.grasp(False, False)

Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.63)


### Test motion

In [15]:
crob.joint_move_make_sure(crob.home_pose+0.05)

Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.63)


In [15]:
crob.joint_move_make_sure(crob.home_pose)

Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.63)


In [17]:
mplan.reset_log(False)
gtimer.reset()

with gtimer.block("detect"):
    for sname in pscene.subject_name_list:
        if isinstance(pscene.subject_dict[sname], AbstractObject):
            pscene.remove_subject(sname)

    gscene.clear_non_fixed()

    ## detect again
    gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])

    ## create boxes
    box_list =  [] 
    for gname in sorted(gtem_dict.keys()):
        box_new = pscene.create_subject(oname=gname, gname=gname, _type=BoxObject, 
                                        hexahedral=True)
        box_new.register_binders(pscene, PlacePlane)
        box_list.append(box_new)

    initial_state = pscene.initialize_state(crob.home_pose, force_fit_binding=True)
    gscene.update_markers_all()

    # remove place points and sub-binders except for the current ones
    use_current_place_point_only(pscene, initial_state)
    use_current_sub_binders_only(pscene, initial_state)
    tplan.prepare()
    mplan.update_gscene()

print(initial_state.node)

gtimer.tic("firstmove")
obj_num = len(gtem_dict)
sweep_num = len(sweep_list)
from_state = initial_state
t_exe = None
snode_schedule_all = []
for sweep_idx in range(sweep_num):
    gcheck.put_banned = [track_list[sweep_idx][2]]
    sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(sweep_num)])
#     sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(2)])+(0,)
    goal_nodes = [("track_face",)*obj_num+sweep_goal]
    if sweep_idx < sweep_num-1:
        for i_s in range(obj_num):
            obj_goal = ["track_face"]*obj_num
            obj_goal[i_s] = "grip1"
            goal_nodes += [tuple(obj_goal)+sweep_goal]
    gtimer.tic("plan{}".format(sweep_idx))
    ppline.search(from_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, 
                  timeout_loop=30, multiprocess=False, timeout=0.5, timeout_constrained=2,
                  add_homing=True, post_optimize=False)
    gtimer.toc("plan{}".format(sweep_idx))
    schedules = ppline.tplan.find_schedules(False)
    schedules_sorted = ppline.tplan.sort_schedule(schedules)
    snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])
    snode_schedule_ori = snode_schedule
    snode_schedule_simple = simplify_schedule(pscene, snode_schedule)
    snode_schedule_safe = calculate_safe_schedule(pscene, snode_schedule_simple, 0.5, 1)
#     double_sweep_motions(snode_schedule_safe)
#     snode_schedule = snode_schedule_safe
    snode_schedule = mix_schedule(mplan, snode_schedule_safe)
    from_state = snode_schedule[-1].state
    if t_exe is not None:
        t_exe.join()
        assert ppline.execute_res, "Previous execution failed"
    else:
        gtimer.toc("firstmove")
#     ppline.execute_schedule(snode_schedule,auto_stop=False, 
#     mode_switcher=ModeSwitcher(pscene), 
#                            multiproc=True)
    t_exe = Thread(target=ppline.execute_schedule, args = (snode_schedule,), 
                    kwargs=dict(
                        auto_stop=False, mode_switcher=ModeSwitcher(pscene),
                        multiproc=True, error_stop_deg=35))
    t_exe.start()
    snode_schedule_all.append(snode_schedule)

if t_exe is not None:
    t_exe.join()

if ppline.execute_res:
    crob.stop_tracking()
    crob.joint_move_make_sure(crob.home_pose)
# else:
#     break

('track_face', 'track_face', 0, 0, 0)
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0)
Motion Filer Failure: TaskClearanceChecker
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0) = fail
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
Motion Filer Failure: ReachChecker
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0) = fail
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
Motion Filer Failure: GraspChecker
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0) = fail
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
Motion Filer Failure: ReachChecker
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0) = fail
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
try transition motion
transition mo

joint motion tried: True
result: 7 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0) = success
branching: 7->8 (3.04/30.0 s, steps/err: 40(106.808900833 ms)/4.31279948005e-16)
try: 7 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0)
Motion Filer Failure: TaskClearanceChecker
result: 7 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0) = fail
try: 8 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0)
Motion Filer Failure: TaskClearanceChecker
result: 8 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0) = fail
try: 5 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0)
try transition motion
transition motion tried: False
Motion Plan Failure
result: 5 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0) = fail
try: 7 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
Motion Filer Failur

result: 0 - ('grip1', 'track_face', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0) = fail
try: 1 - ('track_face', 'track_face', 2, 0, 0)->('track_face', 'grip1', 2, 0, 0)
Motion Filer Failure: GraspChecker
result: 1 - ('track_face', 'track_face', 2, 0, 0)->('track_face', 'grip1', 2, 0, 0) = fail
try: 1 - ('track_face', 'track_face', 2, 0, 0)->('track_face', 'grip1', 2, 0, 0)
Motion Filer Failure: GraspChecker
result: 1 - ('track_face', 'track_face', 2, 0, 0)->('track_face', 'grip1', 2, 0, 0) = fail
try: 0 - ('grip1', 'track_face', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0)
Motion Filer Failure: GraspChecker
result: 0 - ('grip1', 'track_face', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0) = fail
try: 1 - ('track_face', 'track_face', 2, 0, 0)->('track_face', 'grip1', 2, 0, 0)
Motion Filer Failure: GraspChecker
result: 1 - ('track_face', 'track_face', 2, 0, 0)->('track_face', 'grip1', 2, 0, 0) = fail
try: 1 - ('track_face', 'track_face', 2, 0, 0)->('track_face', 'grip1', 2, 0, 0)


transition motion tried: True
result: 3 - ('track_face', 'grip1', 2, 2, 2)->('track_face', 'track_face', 2, 2, 2) = success
branching: 3->4 (0.48/30.0 s, steps/err: 44(110.727071762 ms)/0.000805765884781)
++ adding return motion to acquired answer ++
======================= terminated 0: first answer acquired ===============================
binder: track_face
rname: None
binder: track_face
rname: None
binder: brush_face
rname: indy0
binder: None
binder: None
Connect: Server IP (192.168.0.63)
binder: track_face
rname: None
binder: grip1
rname: panda1
binder: brush_face
rname: indy0
binder: None
binder: None
Connect: Server IP (192.168.0.63)
binder: track_face
rname: None
binder: grip1
rname: panda1
binder: brush_face
rname: indy0
binder: brush_face
rname: indy0
binder: None
Connect: Server IP (192.168.0.63)
http://192.168.0.63:9990/param_setting?control_force0=1
http://192.168.0.63:9990/param_setting?control_force0=0
binder: track_face
rname: None
binder: grip1
rname: panda1
binder: bru

In [20]:
crob.stop_tracking()
crob.joint_move_make_sure(crob.home_pose)

Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.63)


In [13]:
for snode_schedule in snode_schedule_all:
    ppline.play_schedule(snode_schedule, period=0.001)

('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0)
('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
('track_face', 'grip1', 0, 0, 0)->('track_face', 'grip1', 1, 0, 0)
('track_face', 'grip1', 1, 0, 0)->('track_face', 'grip1', 2, 0, 0)
('track_face', 'grip1', 2, 0, 0)->('track_face', 'grip1', 2, 0, 0)


## Plan & show

In [13]:
mplan.reset_log(False)
gtimer.reset()

with gtimer.block("detect"):
    for sname in pscene.subject_name_list:
        if isinstance(pscene.subject_dict[sname], AbstractObject):
            pscene.remove_subject(sname)
            
    gscene.clear_non_fixed()
    
    ## detect again
    gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])

    ## create boxes
    box_list =  [] 
    for gname in sorted(gtem_dict.keys()):
        box_new = pscene.create_subject(oname=gname, gname=gname, _type=BoxObject, hexahedral=True)
        box_new.register_binders(pscene, PlacePlane)
        box_list.append(box_new)
    
    initial_state = pscene.initialize_state(crob.home_pose, force_fit_binding=True)
    gscene.update_markers_all()

    # remove place points and sub-binders except for the current ones
    use_current_place_point_only(pscene, initial_state)
    use_current_sub_binders_only(pscene, initial_state)
    tplan.prepare()
    mplan.update_gscene()
    
print(initial_state.node)

gtimer.tic("firstmove")
obj_num = len(gtem_dict)
sweep_num = len(sweep_list)
from_state = initial_state
t_exe = None
snode_schedule_all = []
for sweep_idx in range(sweep_num):
    gcheck.put_banned = [track_list[sweep_idx][2]]
    sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(sweep_num)])
#     sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(2)])+(0,)
    goal_nodes = [("track_face",)*obj_num+sweep_goal]
    if sweep_idx < sweep_num-1:
        for i_s in range(obj_num):
            obj_goal = ["track_face"]*obj_num
            obj_goal[i_s] = "grip1"
            goal_nodes += [tuple(obj_goal)+sweep_goal]
    gtimer.tic("plan{}".format(sweep_idx))
    ppline.search(from_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, 
                  timeout_loop=30, multiprocess=False, timeout=0.5, timeout_constrained=2, add_homing=True, post_optimize=False)
    gtimer.toc("plan{}".format(sweep_idx))
    schedules = ppline.tplan.find_schedules(False)
    schedules_sorted = ppline.tplan.sort_schedule(schedules)
    snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])
    snode_schedule_ori = snode_schedule
    snode_schedule_simple = simplify_schedule(pscene, snode_schedule)
    snode_schedule_safe = calculate_safe_schedule(pscene, snode_schedule_simple, 0.5, 1)
#     double_sweep_motions(snode_schedule_safe)
#     snode_schedule = snode_schedule_safe
    snode_schedule = mix_schedule(mplan, snode_schedule_safe)
    from_state = snode_schedule[-1].state
    if t_exe:
        t_exe.join()
    else:
        if len(snode_schedule_all)==0:
            gtimer.toc("firstmove")
    snode_schedule_all.append(snode_schedule)
#     t_exe = Thread(target=ppline.play_schedule, args = (snode_schedule,), kwargs=dict( period=0.01))
#     t_exe.start()
# t_exe.join()

('track_face', 'track_face', 0, 0, 0)
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0)
Motion Filer Failure: TaskClearanceChecker
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0) = fail
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
Motion Filer Failure: GraspChecker
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0) = fail
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
Motion Filer Failure: ReachChecker
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0) = fail
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
try transition motion
transition motion tried: False
Motion Plan Failure
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0) = fail
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 

constrained motion tried: True
result: 7 - ('track_face', 'grip1', 1, 0, 0)->('track_face', 'grip1', 2, 0, 0) = success
branching: 7->8 (3.3/30.0 s, steps/err: 83(82.2501182556 ms)/0.000862463590321)
++ adding return motion to acquired answer ++
++ post-optimizing acquired answer ++
======================= terminated 0: first answer acquired ===============================
try: 0 - ('track_face', 'grip1', 2, 0, 0)->('track_face', 'grip1', 2, 1, 0)
try transition motion
transition motion tried: True
result: 0 - ('track_face', 'grip1', 2, 0, 0)->('track_face', 'grip1', 2, 1, 0) = success
branching: 0->1 (0.07/30.0 s, steps/err: 33(72.1120834351 ms)/0.00163953884408)
try: 1 - ('track_face', 'grip1', 2, 1, 0)->('track_face', 'grip1', 2, 2, 0)
try constrained motion
constrained motion tried: True
result: 1 - ('track_face', 'grip1', 2, 1, 0)->('track_face', 'grip1', 2, 2, 0) = success
branching: 1->2 (0.13/30.0 s, steps/err: 75(53.092956543 ms)/0.00176050796123)
++ adding return motion to ac

joint motion tried: True
result: 3 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0) = success
branching: 3->4 (3.41/30.0 s, steps/err: 66(106.930017471 ms)/1.07973612262e-15)
try: 3 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 1)
Motion Filer Failure: TaskClearanceChecker
result: 3 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 1) = fail
try: 4 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 1)
Motion Filer Failure: TaskClearanceChecker
result: 4 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 1) = fail
try: 0 - ('track_face', 'grip1', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0)
Motion Filer Failure: GraspChecker
result: 0 - ('track_face', 'grip1', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0) = fail
try: 3 - ('track_face', 'track_face', 2, 2, 0)->('grip1', 'track_face', 2, 2, 0)
Motion Filer Failure: GraspChecker
result: 3 - ('track_fa

In [ ]:
print(gtimer)

In [ ]:
# for i_ss, snode_schedule in enumerate(snode_schedule_all):
#     for i_s, snode in enumerate(snode_schedule):
#         if snode.traj is not None:
#             save_json("data/traj_{}_{}.json".format(i_ss, i_s), snode.traj[:,:6])

## mix full schedule

In [14]:
gtimer.reset()
snode_schedule_cat = [snode_schedule_all[0][0]]
for snode_schedule in snode_schedule_all:
    snode_schedule_cat += snode_schedule[1:]

with gtimer.block("mix_schedule_safe"):
    safe_mixed = mix_schedule(mplan, snode_schedule_cat)

print(gtimer)

mix_schedule_safe: 	145.9 ms/1 = 145.9 ms (145.925/145.925)



### play schedule

In [15]:
ppline.play_schedule(safe_mixed, period=0.001)

('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0)
('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
('track_face', 'grip1', 0, 0, 0)->('track_face', 'grip1', 1, 0, 0)
('track_face', 'grip1', 1, 0, 0)->('track_face', 'grip1', 2, 0, 0)
('track_face', 'grip1', 2, 0, 0)->('track_face', 'grip1', 2, 0, 0)
('track_face', 'grip1', 2, 0, 0)->('track_face', 'grip1', 2, 1, 0)
('track_face', 'grip1', 2, 1, 0)->('track_face', 'grip1', 2, 2, 0)
('track_face', 'grip1', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0)
('track_face', 'track_face', 2, 2, 0)->('grip1', 'track_face', 2, 2, 0)
('grip1', 'track_face', 2, 2, 0)->('grip1', 'track_face', 2, 2, 0)
('grip1', 'track_face', 2, 2, 0)->('grip1', 'track_face', 2, 2, 1)
('grip1', 'track_face', 2, 2, 1)->('grip1', 'track_face', 2, 2, 2)
('grip1', 'track_face', 2, 2, 2)->('track_face', 'track_face', 2, 2, 2)
('track_face', 'track_face', 2, 2, 2)->('track_face', 'track_face', 2, 2, 2)


In [ ]:
# ppline.play_schedule(safe_mixed[:1], period=0.001)
# color_off = (0.8,0.2,0.2,0.2)
# for obj_name in ppline.pscene.subject_name_list:
#     if isinstance(ppline.pscene.subject_dict[obj_name], AbstractTask):
#         actor, obj = ppline.pscene.actor_dict['brush_face'], ppline.pscene.subject_dict[obj_name]
#         for gtem in obj.clearance:
#             gtem.color = color_off
#             ppline.pscene.gscene.update_marker(gtem)
#             ppline.pscene.set_object_state(snode_schedule_all[0][0].state)
# ppline.pscene.gscene.update_markers_all()

In [ ]:
# from pkg.utils.code_scraps import play_schedule_clearance_highlight
# play_schedule_clearance_highlight(ppline, safe_mixed, tcheck=tcheck, period=0.001)

### Run fully mixed schedule

In [7]:
from pkg.planning.constraint.constraint_common import get_binding_margins, fit_binding
crob.reset_connection(True,True)

connection command:
indy0: True
panda1: True
Connect: Server IP (192.168.0.63)


In [8]:
crob.stop_tracking()

Connect: Server IP (192.168.0.63)


In [9]:
crob.grasp(True, True)
time.sleep(0.5)
crob.grasp(False, False)

Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.63)


In [61]:
crob.joint_move_make_sure(crob.home_pose+0.05)

Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.63)


In [10]:
crob.joint_move_make_sure(crob.home_pose)

Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.63)


In [63]:
ppline.execute_schedule(safe_mixed, mode_switcher=ModeSwitcher(pscene),
                        multiproc=True, error_stop_deg=35)

binder: brush_face
rname: indy0
binder: None
binder: None
Connect: Server IP (192.168.0.63)
http://192.168.0.63:9990/param_setting?control_force0=1
http://192.168.0.63:9990/param_setting?control_force0=0
binder: brush_face
rname: indy0
binder: None
binder: None
Connect: Server IP (192.168.0.63)
binder: brush_face
rname: indy0
binder: None
binder: None
Connect: Server IP (192.168.0.63)
binder: brush_face
rname: indy0
binder: brush_face
rname: indy0
binder: None
Connect: Server IP (192.168.0.63)
http://192.168.0.63:9990/param_setting?control_force0=1
http://192.168.0.63:9990/param_setting?control_force0=0
binder: brush_face
rname: indy0
binder: brush_face
rname: indy0
binder: None
Connect: Server IP (192.168.0.63)
binder: brush_face
rname: indy0
binder: brush_face
rname: indy0
binder: None
Connect: Server IP (192.168.0.63)
binder: brush_face
rname: indy0
binder: brush_face
rname: indy0
binder: brush_face
rname: indy0
Connect: Server IP (192.168.0.63)
http://192.168.0.63:9990/param_settin